In [4]:
# Bring in deps
import os 
from lchain.apikey import apikey 



ModuleNotFoundError: No module named 'lchain'

In [ ]:
import streamlit as st 
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain, SimpleSequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 

os.environ['OPENAI_API_KEY'] = apikey

In [31]:
# App framework
prompt = '目前销量最好的品类是哪个？'

data_info = f"""
dataset name: 行业市场变化_市场大盘分析_行业市场大盘_魔镜

dataset description:
primary keys:[月份, 类目编码]
3540 rows
| field | type | uniques | nan_percentage | max | min |
| --- | --- | --- | --- | --- | --- |
| 月份 | DateTime | 59 | 0.0 | nan | nan |
| 平台编码 | String | 2 | 0.0 | nan | nan |
| 一级类目 | String | 4 | 0.0 | nan | nan |
| 二级类目 | String | 8 | 6.666666666666667 | nan | nan |
| 三级类目 | String | 16 | 20.0 | nan | nan |
| 四级类目 | String | 32 | 46.666666666666664 | nan | nan |
| 类目名称 | String | 60 | 0.0 | nan | nan |
| 类目编码 | Numeric | 60 | 0.0 | 39264.0 | 10209.0 |
| 销售额 | Numeric | 3540 | 0.0 | 99989.30547 | 1.8873 |
| 销量 | Numeric | 974 | 0.0 | 999.0 | 0.0 |
| 店铺数 | Numeric | 181 | 0.0 | 200.0 | 20.0 |
| 商品数 | Numeric | 451 | 0.0 | 500.0 | 50.0 |
| 收藏次数 | Numeric | 1 | 0.0 | 0.0 | 0.0 |
"""


In [55]:

template = """
    Question：{prompt}
    你是一个数据分析机器人，用户会向你咨询关于数据的问题，请按照以下步骤进行处理。
        1. 需要判断用户的问题是否为这两类【常规数据咨询，BA看板】，对于这两个问题之外的，请礼貌的回答无法满足要求。
            想知道具体单一数值或简单问题的，为数据咨询；若用户想知道多个数据的占比或趋势或涉及多个数据维度之间的关系，属于BA看板。
        2A. 如果为数据咨询，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以打印出回答此问题的信息。
        2B. 如果为BA看板，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以绘制对应需求。
        3. 以JSON格式返回信息。Ensure the response can be parsed by Python json.loads。Json外不能有任何字符
        "message":<所有需要沟通的内容>, "task":<问题分类>, "python_code":<python code>
    
    要求：写python代码时需假设df已经读取在内存中，你不需要写读取数据的代码。

    你掌握的数据情况如下：{data_info}
        
"""


In [56]:
Task_splitting_template = PromptTemplate(
    input_variables =['data_info', 'prompt'],
    template = template
) 

In [ ]:
template2 = PromptTemplate(
    input_variables =['python_code'],
    template = '基于生成的代码结果，'
) 

In [45]:
llm = OpenAI(temperature=0.9)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)

In [46]:
response = title_chain.run({'prompt':prompt, 'data_info':data_info})



> Entering new LLMChain chain...
Prompt after formatting:

    你是一个数据分析机器人，用户会向你咨询关于数据的问题，请按照以下步骤进行处理。
        1. 需要判断用户的问题是否为这两类【常规数据咨询，BA看板】，对于这两个问题之外的，请礼貌的回答无法满足要求。
            想知道具体单一数值或简单问题的，为数据咨询；若用户想知道多个数据的占比或趋势或涉及多个数据维度之间的关系，属于BA看板。
        2A. 如果为数据咨询，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以打印出回答此问题的信息。
        2B. 如果为BA看板，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以绘制对应需求。
        3. 以JSON格式返回信息。Ensure the response can be parsed by Python json.loads。Json外不能有任何字符
        "message":<所有需要沟通的内容>, "task":<问题分类>, "python_code":<python code>
    这是具体问题：目前销量最好的品类是哪个？

    要求：写python代码时需假设df已经读取在内存中，你不需要写读取数据的代码。

    你掌握的数据情况如下：
dataset name: 行业市场变化_市场大盘分析_行业市场大盘_魔镜

dataset description:
primary keys:[月份, 类目编码]
3540 rows
| field | type | uniques | nan_percentage | max | min |
| --- | --- | --- | --- | --- | --- |
| 月份 | DateTime | 59 | 0.0 | nan | nan |
| 平台编码 | String | 2 | 0.0 | nan | nan |
| 一级类目 | String | 4 | 0.0 | nan | nan |
| 二级类目 | String | 8 | 6.666666666666667 | nan | 

In [47]:
print(response)

    答案：
"message": "目前销量最高的品类是：", "task": "数据咨询", "python_code": "print(df.sort_values('销量', ascending=False).head(1)['类目名称'].iloc[0])"


In [48]:
response

'    答案：\n"message": "目前销量最高的品类是：", "task": "数据咨询", "python_code": "print(df.sort_values(\'销量\', ascending=False).head(1)[\'类目名称\'].iloc[0])"'

In [49]:
response['choices']

TypeError: string indices must be integers